In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#import cartopy.crs as ccrs
import pandas as pd

import time


In [2]:
# - grid file
gridfile = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/coordinates_eNATL60.nc'
gridfile_ORCA1 = '/scratch/cnt0024/hmg2840/albert7a/eORCA1/eORCA1-I/IPSLCM6ALR_eORCA1.2_mesh_mask.nc'

# - Define read data function
def read_datagrid(gridfile,latmin=None,latmax=None,lonmin=None,lonmax=None):
    """Return navlon,navlat."""
    ncfile = Dataset(gridfile,'r')
    # load navlon and navlat
    _navlon = ncfile.variables['glamt'][:,:]
    _navlat = ncfile.variables['gphit'][:,:]
    #-Define domain
    domain = (lonmin<_navlon) * (_navlon<lonmax) * (latmin<_navlat) * (_navlat<latmax)
    where = np.where(domain)
    vlats = _navlat[where]
    vlons = _navlon[where]
    #get indice
    jmin = where[0][vlats.argmin()]
    jmax = where[0][vlats.argmax()]
    imin = where[1][vlons.argmin()]
    imax = where[1][vlons.argmax()]
    #load arrays
    navlon = _navlon[jmin:jmax+1,imin:imax+1]
    navlat = _navlat[jmin:jmax+1,imin:imax+1]
    return navlon,navlat,jmin,jmax,imin,imax

def read_datagrid_ORCA1(gridfile,latmin=None,latmax=None,lonmin=None,lonmax=None):
    """Return navlon,navlat."""
    ncfile = Dataset(gridfile,'r')
    # load navlon and navlat
    _navlon = ncfile.variables['nav_lon'][:,:]
    _navlat = ncfile.variables['nav_lat'][:,:]
    #-Define domain
    domain = (lonmin<_navlon) * (_navlon<lonmax) * (latmin<_navlat) * (_navlat<latmax)
    where = np.where(domain)
    vlats = _navlat[where]
    vlons = _navlon[where]
    #get indice
    jmin_ORCA = where[0][vlats.argmin()]
    jmax_ORCA = where[0][vlats.argmax()]
    imin_ORCA = where[1][vlons.argmin()]
    imax_ORCA = where[1][vlons.argmax()]
    #load arrays
    navlon_ORCA = _navlon[jmin_ORCA:jmax_ORCA+1,imin_ORCA:imax_ORCA+1]
    navlat_ORCA = _navlat[jmin_ORCA:jmax_ORCA+1,imin_ORCA:imax_ORCA+1]
    return navlon_ORCA,navlat_ORCA,jmin_ORCA,jmax_ORCA,imin_ORCA,imax_ORCA


In [3]:
#Define region 
y1 = 55. 
y2 = 62.
x1 = -49.
x2 = -56.

navlon_ORCA,navlat_ORCA,jmin_ORCA,jmax_ORCA,imin_ORCA,imax_ORCA = read_datagrid_ORCA1(gridfile_ORCA1,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)


In [33]:
navlon_ORCA.shape

(13, 8)

In [35]:
np.shape(navlat_ORCA)[1]-2.

6.0

In [22]:
ncfile = Dataset(gridfile_ORCA1,'r')
# load navlon and navlat
_navlon = ncfile.variables['nav_lon'][:,:]
_navlat = ncfile.variables['nav_lat'][:,:]


In [8]:
print('jmin =',jmin_ORCA,'jmax =',jmax_ORCA,'imin =',imin_ORCA,'imax =',imax_ORCA)


jmin = 267 jmax = 279 imin = 234 imax = 241


In [19]:
truc=navlat_ORCA.flatten()
len(truc)

104

In [37]:
navlat_ORCA[1,0]

55.30585

In [38]:
navlat_ORCA[0,0]

54.724823

In [36]:
np.abs(navlat_ORCA[1,0] - navlat_ORCA[0,0])/2.

0.2905139923095703

In [43]:
list_i = []
list_j = []
list_name = []
list_lat = []
list_lon = []
list_lat_eORCA1 = []
list_lon_eORCA1 = []

a = (np.shape(navlon_ORCA)[0]-2.)*(np.shape(navlat_ORCA)[1]-2.)

ii = 0
jj = 0
for i in np.arange(0,int(a),1):
    ii = ii + 1
    if ii >= int(np.shape(navlon_ORCA)[0])-2:
        ii = 0
        jj = jj + 1
    list_i.append(imin_ORCA+ii)
    list_j.append(jmin_ORCA+jj)
    list_lat.append(navlat_ORCA[ii,jj])
    list_lon.append(navlon_ORCA[ii,jj+1])
    list_lat_eORCA1.append(_navlat[jmin_ORCA+jj+1,imin_ORCA+ii+1])
    list_lon_eORCA1.append(_navlon[jmin_ORCA+jj+1,imin_ORCA+ii+1])
    list_name.append('Box_'+str(i))


In [44]:
panda_box=pd.DataFrame({'i' : list_i, 'j' : list_j, 'lat' : list_lat, 'lon' : list_lon, 'lat_eORCA1' : list_lat_eORCA1, 'lon_eORCA1' : list_lon_eORCA1},index = list_name)
panda_box.to_csv('boxes_LS_1x1_eORCA1.csv',sep = '\t')

In [5]:


# - Define box dimensions
a = (np.shape(navlon_ORCA)[0]-2.)*(np.shape(navlat_ORCA)[1]-2.)
Box_arr = np.empty((int(a),6),dtype=np.dtype('U100'))
print(a)

ii = 0
jj = 0
for i in np.arange(0,int(a),1):
    delta_lat1 = np.abs(navlat_ORCA[ii+1,jj] - navlat_ORCA[ii,jj])/2.
    delta_lat2 = np.abs(navlat_ORCA[ii+2,jj] - navlat_ORCA[ii+1,jj])/2.
    delta_lon1 = np.abs(navlon_ORCA[ii,jj+1] - navlon_ORCA[ii,jj])/2.
    delta_lon2 = np.abs(navlon_ORCA[ii,jj+2] - navlon_ORCA[ii,jj+1])/2.    
    Box_arr[i,:] = [str(navlat_ORCA[ii,jj]+delta_lat1),str(navlat_ORCA[ii+1,jj]+delta_lat2),str(navlon_ORCA[ii,jj]+delta_lon1),str(navlon_ORCA[ii,jj+1]+delta_lon2),'Box_'+str(i),str(i)]
    ii = ii + 1
    if ii >= int(np.shape(navlon_ORCA)[0])-2:
        ii = 0
        jj = jj + 1

name = Box_arr[:,4]

list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []

for i in np.arange(0,int(a),1):
    
    y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
    x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
    box_name = Box_arr[i,4]
    print(box_name)
    
    # - Obtain navlon and Navlat
    navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)
    
    list_imin.append(imin)
    list_imax.append(imax)
    list_jmin.append(jmin)
    list_jmax.append(jmax)
    list_name.append(box_name)
    list_latmin.append(y1)
    list_latmax.append(y2)
    list_lonmin.append(x1)
    list_lonmax.append(x2)
    
panda_box=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax },index = list_name)
panda_box.to_csv('boxes_LS_1x1_eNATL60.csv',sep = '\t')

66.0
Box_0
Box_1
Box_2
Box_3
Box_4
Box_5
Box_6
Box_7
Box_8
Box_9
Box_10
Box_11
Box_12
Box_13
Box_14
Box_15
Box_16
Box_17
Box_18
Box_19
Box_20
Box_21
Box_22
Box_23
Box_24
Box_25
Box_26
Box_27
Box_28
Box_29
Box_30
Box_31
Box_32
Box_33
Box_34
Box_35
Box_36
Box_37
Box_38
Box_39
Box_40
Box_41
Box_42
Box_43
Box_44
Box_45
Box_46
Box_47
Box_48
Box_49
Box_50
Box_51
Box_52
Box_53
Box_54
Box_55
Box_56
Box_57
Box_58
Box_59
Box_60
Box_61
Box_62
Box_63
Box_64
Box_65


In [7]:
#Define region
y1 = 32.5 
y2 = 41.5
x1 = -59.
x2 = -71.6

navlon_ORCA,navlat_ORCA,jmin_ORCA,jmax_ORCA,imin_ORCA,imax_ORCA = read_datagrid_ORCA1(gridfile_ORCA1,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)

# - Define box dimensions
a = (np.shape(navlon_ORCA)[0]-2.)*(np.shape(navlat_ORCA)[1]-2.)
Box_arr = np.empty((int(a),6),dtype=np.dtype('U100'))
print(a)

ii = 0
jj = 0
for i in np.arange(0,int(a),1):
    delta_lat1 = np.abs(navlat_ORCA[ii+1,jj] - navlat_ORCA[ii,jj])/2.
    delta_lat2 = np.abs(navlat_ORCA[ii+2,jj] - navlat_ORCA[ii+1,jj])/2.
    delta_lon1 = np.abs(navlon_ORCA[ii,jj+1] - navlon_ORCA[ii,jj])/2.
    delta_lon2 = np.abs(navlon_ORCA[ii,jj+2] - navlon_ORCA[ii,jj+1])/2.    
    Box_arr[i,:] = [str(navlat_ORCA[ii,jj]+delta_lat1),str(navlat_ORCA[ii+1,jj]+delta_lat2),str(navlon_ORCA[ii,jj]+delta_lon1),str(navlon_ORCA[ii,jj+1]+delta_lon2),'Box_'+str(i),str(i)]
    ii = ii + 1
    if ii >= int(np.shape(navlon_ORCA)[0])-2:
        ii = 0
        jj = jj + 1

name = Box_arr[:,4]


list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []

for i in np.arange(0,int(a),1):

    y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
    x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
    box_name = Box_arr[i,4]
    print(box_name)
    
    # - Obtain navlon and Navlat
    navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)

    list_imin.append(imin)
    list_imax.append(imax)
    list_jmin.append(jmin)
    list_jmax.append(jmax)
    list_name.append(box_name)
    list_latmin.append(y1)
    list_latmax.append(y2)
    list_lonmin.append(x1)
    list_lonmax.append(x2)
    
panda_box_GS=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)
panda_box_GS.to_csv('boxes_GS_1x1_eNATL60.csv',sep = '\t')

110.0
Box_0
Box_1
Box_2
Box_3
Box_4
Box_5
Box_6
Box_7
Box_8
Box_9
Box_10
Box_11
Box_12
Box_13
Box_14
Box_15
Box_16
Box_17
Box_18
Box_19
Box_20
Box_21
Box_22
Box_23
Box_24
Box_25
Box_26
Box_27
Box_28
Box_29
Box_30
Box_31
Box_32
Box_33
Box_34
Box_35
Box_36
Box_37
Box_38
Box_39
Box_40
Box_41
Box_42
Box_43
Box_44
Box_45
Box_46
Box_47
Box_48
Box_49
Box_50
Box_51
Box_52
Box_53
Box_54
Box_55
Box_56
Box_57
Box_58
Box_59
Box_60
Box_61
Box_62
Box_63
Box_64
Box_65
Box_66
Box_67
Box_68
Box_69
Box_70
Box_71
Box_72
Box_73
Box_74
Box_75
Box_76
Box_77
Box_78
Box_79
Box_80
Box_81
Box_82
Box_83
Box_84
Box_85
Box_86
Box_87
Box_88
Box_89
Box_90
Box_91
Box_92
Box_93
Box_94
Box_95
Box_96
Box_97
Box_98
Box_99
Box_100
Box_101
Box_102
Box_103
Box_104
Box_105
Box_106
Box_107
Box_108
Box_109


In [ ]:
y1 = 41. 
y2 = 56.
x1 = -19.
x2 = -41.

navlon_ORCA,navlat_ORCA,jmin_ORCA,jmax_ORCA,imin_ORCA,imax_ORCA = read_datagrid_ORCA1(gridfile_ORCA1,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)

# - Define box dimensions
a = (np.shape(navlon_ORCA)[0]-2.)*(np.shape(navlat_ORCA)[1]-2.)
Box_arr = np.empty((int(a),6),dtype=np.dtype('U100'))
print(a)

ii = 0
jj = 0
for i in np.arange(0,int(a),1):
    delta_lat1 = np.abs(navlat_ORCA[ii+1,jj] - navlat_ORCA[ii,jj])/2.
    delta_lat2 = np.abs(navlat_ORCA[ii+2,jj] - navlat_ORCA[ii+1,jj])/2.
    delta_lon1 = np.abs(navlon_ORCA[ii,jj+1] - navlon_ORCA[ii,jj])/2.
    delta_lon2 = np.abs(navlon_ORCA[ii,jj+2] - navlon_ORCA[ii,jj+1])/2.        
    Box_arr[i,:] = [str(navlat_ORCA[ii,jj]+delta_lat1),str(navlat_ORCA[ii+1,jj]+delta_lat2),str(navlon_ORCA[ii,jj]+delta_lon1),str(navlon_ORCA[ii,jj+1]+delta_lon2),'Box_'+str(i),str(i)]
    ii = ii + 1
    if ii >= int(np.shape(navlon_ORCA)[0])-2:
        ii = 0
        jj = jj + 1
        
name = Box_arr[:,4]

list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []

i=0

y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
box_name = Box_arr[i,4]
print(box_name)
    
# - Obtain navlon and Navlat
navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)
    
list_imin.append(imin)
list_imax.append(imax)
list_jmin.append(jmin)
list_jmax.append(jmax)
list_name.append(box_name)
list_latmin.append(y1)
list_latmax.append(y2)
list_lonmin.append(x1)
list_lonmax.append(x2)

panda_box_MNA=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)
panda_box_MNA.to_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t')

list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []

for i in np.arange(1,200,1):
    
    y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
    x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
    box_name = Box_arr[i,4]
    print(box_name)
    
    # - Obtain navlon and Navlat
    navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)
    
    list_imin.append(imin)
    list_imax.append(imax)
    list_jmin.append(jmin)
    list_jmax.append(jmax)
    list_name.append(box_name)
    list_latmin.append(y1)
    list_latmax.append(y2)
    list_lonmin.append(x1)
    list_lonmax.append(x2)
    
panda_box_MNA=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)
panda_box_MNA_1=pd.read_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t',index_col=0)
panda_box_MNA_2=panda_box_MNA_1.append(panda_box_MNA)
panda_box_MNA_2.to_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t')

list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []

for i in np.arange(200,int(a),1):
    
    y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
    x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
    box_name = Box_arr[i,4]
    print(box_name)
    
    # - Obtain navlon and Navlat
    navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)
    
    list_imin.append(imin)
    list_imax.append(imax)
    list_jmin.append(jmin)
    list_jmax.append(jmax)
    list_name.append(box_name)
    list_latmin.append(y1)
    list_latmax.append(y2)
    list_lonmin.append(x1)
    list_lonmax.append(x2)

panda_box_MNA=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)
panda_box_MNA_2=pd.read_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t',index_col=0)
panda_box_MNA_3=panda_box_MNA_2.append(panda_box_MNA)
panda_box_MNA_3.to_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t')


400.0
Box_0
Box_1
Box_2
Box_3
Box_4
Box_5
Box_6
Box_7
Box_8
Box_9
Box_10
Box_11
Box_12
Box_13
Box_14
Box_15
Box_16
Box_17
Box_18
Box_19
Box_20
Box_21
Box_22
Box_23
Box_24
Box_25
Box_26
Box_27
Box_28
Box_29
Box_30
Box_31
Box_32
Box_33
Box_34
Box_35
Box_36
Box_37
Box_38
Box_39
Box_40
Box_41
Box_42
Box_43
Box_44
Box_45
Box_46
Box_47
Box_48
Box_49
Box_50
Box_51
Box_52
Box_53
Box_54
Box_55
Box_56
Box_57
Box_58
Box_59
Box_60
Box_61
Box_62
Box_63
Box_64
Box_65
Box_66
Box_67
Box_68
Box_69
Box_70
Box_71
Box_72
Box_73
Box_74
Box_75
Box_76
Box_77
Box_78
Box_79
Box_80
Box_81
Box_82
Box_83
Box_84
Box_85
Box_86
Box_87
Box_88
Box_89
Box_90
Box_91
Box_92
Box_93
Box_94
Box_95
Box_96
Box_97
Box_98
Box_99
Box_100
Box_101
Box_102
Box_103
Box_104
Box_105
Box_106
Box_107
Box_108
Box_109
Box_110
Box_111
Box_112
Box_113
Box_114
Box_115
Box_116
Box_117
Box_118
Box_119
Box_120
Box_121
Box_122
Box_123
Box_124
Box_125
Box_126
Box_127
Box_128
Box_129
Box_130
Box_131
Box_132
Box_133
Box_134
Box_135
Box_136
Box_137


In [3]:
y1 = 41. 
y2 = 56.
x1 = -19.
x2 = -41.

navlon_ORCA,navlat_ORCA,jmin_ORCA,jmax_ORCA,imin_ORCA,imax_ORCA = read_datagrid_ORCA1(gridfile_ORCA1,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)

# - Define box dimensions
a = (np.shape(navlon_ORCA)[0]-2.)*(np.shape(navlat_ORCA)[1]-2.)
Box_arr = np.empty((int(a),6),dtype=np.dtype('U100'))
print(a)

ii = 0
jj = 0
for i in np.arange(0,int(a),1):
    delta_lat1 = np.abs(navlat_ORCA[ii+1,jj] - navlat_ORCA[ii,jj])/2.
    delta_lat2 = np.abs(navlat_ORCA[ii+2,jj] - navlat_ORCA[ii+1,jj])/2.
    delta_lon1 = np.abs(navlon_ORCA[ii,jj+1] - navlon_ORCA[ii,jj])/2.
    delta_lon2 = np.abs(navlon_ORCA[ii,jj+2] - navlon_ORCA[ii,jj+1])/2.        
    Box_arr[i,:] = [str(navlat_ORCA[ii,jj]+delta_lat1),str(navlat_ORCA[ii+1,jj]+delta_lat2),str(navlon_ORCA[ii,jj]+delta_lon1),str(navlon_ORCA[ii,jj+1]+delta_lon2),'Box_'+str(i),str(i)]
    ii = ii + 1
    if ii >= int(np.shape(navlon_ORCA)[0])-2:
        ii = 0
        jj = jj + 1
        
name = Box_arr[:,4]

list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []

#i=0

#y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
#x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
#box_name = Box_arr[i,4]
#print(box_name)
    
# - Obtain navlon and Navlat
#navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)
    
#list_imin.append(imin)
#list_imax.append(imax)
#list_jmin.append(jmin)
#list_jmax.append(jmax)
#list_name.append(box_name)
#list_latmin.append(y1)
#list_latmax.append(y2)
#list_lonmin.append(x1)
#list_lonmax.append(x2)

#panda_box_MNA=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)
#panda_box_MNA.to_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t')

#list_imin = []
#list_imax = []
#list_jmin = []
#list_jmax = []
#list_name = []
#list_latmin = []
#list_latmax = []
#list_lonmin = []
#list_lonmax = []

#for i in np.arange(1,200,1):
    
#    y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
#    x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
#    box_name = Box_arr[i,4]
#    print(box_name)
    
    # - Obtain navlon and Navlat
#    navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)
    
#    list_imin.append(imin)
#    list_imax.append(imax)
#    list_jmin.append(jmin)
#    list_jmax.append(jmax)
#    list_name.append(box_name)
#    list_latmin.append(y1)
#    list_latmax.append(y2)
#    list_lonmin.append(x1)
#    list_lonmax.append(x2)
    
#panda_box_MNA=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)
#panda_box_MNA_1=pd.read_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t',index_col=0)
#panda_box_MNA_2=panda_box_MNA_1.append(panda_box_MNA)
#panda_box_MNA_2.to_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t')

list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []

for i in np.arange(200,int(a),1):
    
    y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
    x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
    box_name = Box_arr[i,4]
    print(box_name)
    
    # - Obtain navlon and Navlat
    navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)
    
    list_imin.append(imin)
    list_imax.append(imax)
    list_jmin.append(jmin)
    list_jmax.append(jmax)
    list_name.append(box_name)
    list_latmin.append(y1)
    list_latmax.append(y2)
    list_lonmin.append(x1)
    list_lonmax.append(x2)

panda_box_MNA=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)
panda_box_MNA_2=pd.read_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t',index_col=0)
panda_box_MNA_3=panda_box_MNA_2.append(panda_box_MNA)
panda_box_MNA_3.to_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t')


400.0
Box_200
Box_201
Box_202
Box_203
Box_204
Box_205
Box_206
Box_207
Box_208
Box_209
Box_210
Box_211
Box_212
Box_213
Box_214
Box_215
Box_216
Box_217
Box_218
Box_219
Box_220
Box_221
Box_222
Box_223
Box_224
Box_225
Box_226
Box_227
Box_228
Box_229
Box_230
Box_231
Box_232
Box_233
Box_234
Box_235
Box_236
Box_237
Box_238
Box_239
Box_240
Box_241
Box_242
Box_243
Box_244
Box_245
Box_246
Box_247
Box_248
Box_249
Box_250
Box_251
Box_252
Box_253
Box_254
Box_255
Box_256
Box_257
Box_258
Box_259
Box_260
Box_261
Box_262
Box_263
Box_264
Box_265
Box_266
Box_267
Box_268
Box_269
Box_270
Box_271
Box_272
Box_273
Box_274
Box_275
Box_276
Box_277
Box_278
Box_279
Box_280
Box_281
Box_282
Box_283
Box_284
Box_285
Box_286
Box_287
Box_288
Box_289
Box_290
Box_291
Box_292
Box_293
Box_294
Box_295
Box_296
Box_297
Box_298
Box_299
Box_300
Box_301
Box_302
Box_303
Box_304
Box_305
Box_306
Box_307
Box_308
Box_309
Box_310
Box_311
Box_312
Box_313
Box_314
Box_315
Box_316
Box_317
Box_318
Box_319
Box_320
Box_321
Box_322
Box_323
Bo

In [3]:
y1 = 41. 
y2 = 56.
x1 = -19.
x2 = -41.

navlon_ORCA,navlat_ORCA,jmin_ORCA,jmax_ORCA,imin_ORCA,imax_ORCA = read_datagrid_ORCA1(gridfile_ORCA1,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)

# - Define box dimensions
a = (np.shape(navlon_ORCA)[0]-2.)*(np.shape(navlat_ORCA)[1]-2.)
Box_arr = np.empty((int(a),6),dtype=np.dtype('U100'))
print(a)

ii = 0
jj = 0
for i in np.arange(0,int(a),1):
    delta_lat1 = np.abs(navlat_ORCA[ii+1,jj] - navlat_ORCA[ii,jj])/2.
    delta_lat2 = np.abs(navlat_ORCA[ii+2,jj] - navlat_ORCA[ii+1,jj])/2.
    delta_lon1 = np.abs(navlon_ORCA[ii,jj+1] - navlon_ORCA[ii,jj])/2.
    delta_lon2 = np.abs(navlon_ORCA[ii,jj+2] - navlon_ORCA[ii,jj+1])/2.        
    Box_arr[i,:] = [str(navlat_ORCA[ii,jj]+delta_lat1),str(navlat_ORCA[ii+1,jj]+delta_lat2),str(navlon_ORCA[ii,jj]+delta_lon1),str(navlon_ORCA[ii,jj+1]+delta_lon2),'Box_'+str(i),str(i)]
    ii = ii + 1
    if ii >= int(np.shape(navlon_ORCA)[0])-2:
        ii = 0
        jj = jj + 1
        
name = Box_arr[:,4]

list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []


400.0


In [4]:
i=0
y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
box_name = Box_arr[i,4]
print(box_name)    
# - Obtain navlon and Navlat
navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)    
list_imin.append(imin)
list_imax.append(imax)
list_jmin.append(jmin)
list_jmax.append(jmax)
list_name.append(box_name)
list_latmin.append(y1)
list_latmax.append(y2)
list_lonmin.append(x1)
list_lonmax.append(x2)


Box_0


In [5]:
panda_box_MNA=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)
panda_box_MNA.to_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t')

In [6]:
i=1
list_imin = []
list_imax = []
list_jmin = []
list_jmax = []
list_name = []
list_latmin = []
list_latmax = []
list_lonmin = []
list_lonmax = []
y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
box_name = Box_arr[i,4]
print(box_name)    
# - Obtain navlon and Navlat
navlon,navlat,jmin,jmax,imin,imax = read_datagrid(gridfile,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)    
list_imin.append(imin)
list_imax.append(imax)
list_jmin.append(jmin)
list_jmax.append(jmax)
list_name.append(box_name)
list_latmin.append(y1)
list_latmax.append(y2)
list_lonmin.append(x1)
list_lonmax.append(x2)


Box_1


In [7]:
panda_box_MNA=pd.DataFrame({'imin' : list_imin, 'imax' : list_imax,'jmin' : list_jmin, 'jmax' : list_jmax, 'latmin' : list_latmin, 'latmax' : list_latmax,'lonmin' : list_lonmin, 'lonmax' : list_lonmax},index = list_name)


In [8]:
panda_box_MNA_csv=pd.read_csv('boxes_MNA_1x1_eNATL60.csv',sep = '\t',index_col=0)
panda_box_MNA_bis=panda_box_MNA_csv+panda_box_MNA

In [11]:
panda_box_MNA_csv


,imin,imax,jmin,jmax,latmin,latmax,lonmin,lonmax
Box_0,3474,3534,2349,2409,41.451969,42.211447,-39.094934,-40.102634


In [12]:
panda_box_MNA

,imin,imax,jmin,jmax,latmin,latmax,lonmin,lonmax
Box_1,3474,3534,2409,2469,42.211447,42.963459,-39.1105,-40.119429


In [14]:
panda_box_MNA_csv.append(panda_box_MNA)

,imin,imax,jmin,jmax,latmin,latmax,lonmin,lonmax
Box_0,3474,3534,2349,2409,41.451969,42.211447,-39.094934,-40.102634
Box_1,3474,3534,2409,2469,42.211447,42.963459,-39.110500,-40.119429
